# Borrower reliability study

<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Project-description" data-toc-modified-id="Project-description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Project description</a></span></li><li><span><a href="#Project-progress" data-toc-modified-id="Project-progress-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Project progress</a></span><ul class="toc-item"><li><span><a href="#Data-review" data-toc-modified-id="Data-review-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Data review</a></span></li><li><span><a href="#Filling-in-the-missing-values" data-toc-modified-id="Filling-in-the-missing-values-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Filling in the missing values</a></span></li><li><span><a href="#Checking-data-for-anomalies-and-corrections." data-toc-modified-id="Checking-data-for-anomalies-and-corrections.-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Checking data for anomalies and corrections.</a></span></li><li><span><a href="#Changing-data-types" data-toc-modified-id="Changing-data-types-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Changing data types</a></span></li><li><span><a href="#Removing-duplicates." data-toc-modified-id="Removing-duplicates.-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Removing duplicates.</a></span></li><li><span><a href="#Generation-of-additional-dataframes,-decomposition-of-the-original-dataframe." data-toc-modified-id="Generation-of-additional-dataframes,-decomposition-of-the-original-dataframe.-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Generation of additional dataframes, decomposition of the original dataframe.</a></span></li><li><span><a href="#Income-categorization" data-toc-modified-id="Income-categorization-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Income categorization</a></span></li><li><span><a href="#Credit-purpose-categorization." data-toc-modified-id="Credit-purpose-categorization.-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Credit purpose categorization.</a></span></li><li><span><a href="#Answers-to-questions." data-toc-modified-id="Answers-to-questions.-2.9"><span class="toc-item-num">2.9&nbsp;&nbsp;</span>Answers to questions.</a></span></li></ul></li><li><span><a href="#Overall-conclusion" data-toc-modified-id="Overall-conclusion-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Overall conclusion</a></span></li></ul></div>

## Project description

**Scope of the study**

The client is the credit department of a bank. It needs to find out whether the client's marital status and number of children affects the repayment of the loan on time. Input data from the bank - statistics on clients' repayment capacity.

The results of the study will be taken into account when building a credit scoring model - a special system, which assesses the ability of a potential borrower to repay the loan to the bank.

**Hypothesis testing**

The study will test the hypotheses about the interdependence of individual factors affecting loan repayment by answering the following questions:

- Is there a relationship between the number of children and loan repayment on time?
- Is there a correlation between marital status and loan repayment on time?
- Is there a correlation between income level and repayment on time?
- How do the different purposes of the loan affect repayment on time?


## Project progress

### Data review

In [1]:
import pandas as pd

df = pd.read_csv('/datasets/data.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Examination of the overall data information shows that there are two columns with missing data, namely missing data in the columns: `days_employed` and `total_income`. Let us analyse these columns in more detail.

### Filling in the missing values

In [2]:
# calculate the number of missing values and  
# the proportion of missing values in the total number of values by column in the data frame.

df.isna().agg(['sum', 'mean'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
sum,0.0,2174.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2174.000000,0.0
mean,0.0,0.100999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100999,0.0


We have confirmed our initial finding of missing values in these two columns, we have now calculated the total number of omissions by each column. We also analyse the proportion of missing values in the total number of values in these columns.

The percentage of missing values is almost 10%, which is a very significant value. Missing values up to 1% of the total number of values can be neglected depending on the situation, but with such a high number of missing values certain actions have to be taken to fill them.

In order to assess the omission handling mechanism, it is necessary to understand the nature of the missing values - whether the missing values are completely random, accidental or non-random. In our situation, missing values in the `total_income` column are likely due to the possible reluctance of borrowers to report their income when applying for a loan (especially high-income clients). However, on a cursory superficial analysis, we see that the number of missing values in both columns is exactly the same = 2174. Most likely these omissions appear in the same rows, which could indicate a technical error in the data handling. Below we check if the missing data in both columns actually appear in the same rows, i.e. we check the two columns for symmetry.

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера:</b> Молодец, что высчитал долю! Действительно, она велика для удаления

In [3]:
# checking the 'days_employed' and 'total_income' columns for missing values symmetry

df[df['days_employed'].isna() == True]['total_income'].isna().sum()

2174

For further analysis and work with the dataframe, it is necessary to fill these missing values. Since the values in both columns are quantitative, the most appropriate way to fill such omissions is to fill them with the most characteristic value for that column, either the arithmetic mean or the median value. Since the variation in both columns may be too great, highly outliers may significantly increase or decrease the arithmetic mean. Therefore, in situations where there are outstanding values in the sample, it is better to use the median to fill in the missing values.

At the same time, in this situation, it would not be correct to fill all missing values with a single median value for the entire column, because in our case, years of experience and income are highly correlated with age and, to a lesser extent, with gender. Therefore, it makes much more sense to fill in the missing values with median value in terms of the relevant grouping - in terms of age and gender of the borrower.

To do this, we create a pivot_table that groups the dataframe by age and gender of the borrower and calculates the median value of years of experience for each segment.

In [4]:
# Creating a pivot_table to calculate the borrower's median length of service, by age and gender

pivot_table_days_employed = df.pivot_table(
    index='dob_years', 
    columns='gender', 
    values='days_employed', 
    aggfunc='median')

In [5]:
pivot_table_days_employed.head()

gender,F,M,XNA
dob_years,,,
0,-1327.579211,-1015.395451,NaN
19,-710.230781,-885.268574,NaN
20,-595.559207,-880.528819,NaN
21,-608.003610,-627.185770,NaN
22,-691.825293,-740.851560,NaN


We have calculated the median value separately by age and gender of the borrower. We found an unknown gender - neither male nor female. We will deal with it further in the anomaly processing step. Same as with null age and negative employment experience.

Next, we need to fill in the missing data in the `days_employed` column with the calculated median values. To do this, we will write and apply to our dataframe a function that will go through all rows and fill in the missing values with the corresponding median value.

In [6]:
def days_employed_fillna(dataframe):
    
    """
    The function returns the median value of employment experience
    according to gender and age, if it detects a missing NaN value. 
    Otherwise, it returns the same value.
    """
    gender = dataframe['gender']
    age = dataframe['dob_years']
    if pd.isna(dataframe['days_employed']):
        return pivot_table_days_employed.loc[age,gender]
    else:
        return dataframe['days_employed']

In [7]:
# apply the function to the data frame and fill in the missing values 

df['days_employed'] = df.apply(days_employed_fillna, axis = 1)

In [8]:
# checking the filling in the missing values in days_employed column

df['days_employed'].isna().sum()

0

The missing values in `days_employed` column has been filled in. Now apply the same sequence to the `total_income` column.

In [9]:
# generating a pivot table to calculate the median value of the borrower's total income, by age and gender

pivot_table_total_income = df.pivot_table(
    index='dob_years', 
    columns='gender', 
    values='total_income', 
    aggfunc='median')

In [10]:
pivot_table_total_income.head()

gender,F,M,XNA
dob_years,,,
0,117279.001914,179138.212343,NaN
19,112217.756662,91876.754772,NaN
20,101512.989536,134750.749515,NaN
21,112778.853259,147812.788085,NaN
22,111633.034748,158258.854591,NaN


In [11]:
def total_income_fillna(dataframe):
    
    """
    The function returns the median value of total income
    according to gender and age, if it detects a missing NaN value. 
    Otherwise, it returns the same value.
    """
    gender = dataframe['gender']
    age = dataframe['dob_years']
    if pd.isna(dataframe['total_income']):
        return pivot_table_total_income.loc[age,gender]
    else:
        return dataframe['total_income']

In [12]:
# applying the function to the data frame and fill in the missing values 

df['total_income'] = df.apply(total_income_fillna, axis = 1)

In [13]:
# checking the filling in the missing values in `total_income`

df['total_income'].isna().sum()

0

All missing values are filled in and we can continue preprocessing the data in the table, proceeding to check the data for anomalies.

### Checking data for anomalies and corrections.

To detect possible anomalies in the data, we can analyse the overall distribution of values in the data frame using the `describe()` function.

In [14]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,62976.295708,43.293380,0.817236,0.972544,0.080883,1.651428e+05
std,1.381587,140286.252454,12.574584,0.548138,1.420324,0.272661,9.806554e+04
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2518.168900,33.000000,1.000000,0.000000,0.000000,1.074857e+05
50%,0.000000,-1265.117765,42.000000,1.000000,0.000000,0.000000,1.433694e+05
75%,1.000000,-311.183346,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Processing the column with the number of children of the borrower: `children`**

The presence of a negative value is immediately apparent, as well as the presence of 20 children in the `children` column.

The negative number of children is most likely a technical error - when writing "1" the value "-1" was written. We'll correct this error by replacing "-1" with "1". Just in case, let's check how many entries there are in the table with the number of children "-1" (see the check below). In addition to the negative number of children, the fact that there are 20 children is doubtful, which is very unlikely. Most likely there was a technical error in uploading the data and an extra zero was glued to value 2. We will also correct this error by replacing "20" with "2". Also check the number of erroneous entries in the table with value "20" beforehand

In [15]:
df[df['children'] == -1]['children'].count() # counting rows with the number of children "-1"

47

In [16]:
df[df['children'] == 20]['children'].count() # counting rows with the number of children "20"

76

We see that there are only 47 and 76 such entries with the number of children "-1" and "20", respectively, out of the total number of 21525 rows in the table. Even if the logic of our correction turns out to be wrong, it will not affect the further analysis in any way. Replace "-1" with "1" and "20" with "2".

In [17]:
df['children'] = df['children'].replace(-1,1)

In [18]:
df['children'] = df['children'].replace(20,2)

**Processing the borrower's employment history column: `days_employed`**

It is also obvious that most of the `days_employed` column values are negative, which may be due to a technical error during data uploading that caused a sign change.

Naturally, these artifacts need to be corrected. Let's replace all negative values with their module with the function `.abs()`

In [19]:
df['days_employed'] = df['days_employed'].abs()

In [20]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.479721,66708.046624,43.293380,0.817236,0.972544,0.080883,1.651428e+05
std,0.755528,138550.570118,12.574584,0.548138,1.420324,0.272661,9.806554e+04
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,977.995667,33.000000,1.000000,0.000000,0.000000,1.074857e+05
50%,0.000000,2041.666104,42.000000,1.000000,0.000000,0.000000,1.433694e+05
75%,1.000000,5317.759754,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


We got rid of negative values, which was clearly an anomaly for these columns.

However, we found inadequate minimum and maximum years of service in days (24 and 401755 respectively). Generally, the bank's main requirement for granting credit is to have at least some minimum number of years of experience. Let's assume that the bank's regulations stipulate a minimum of six months of work experience, i.e. 180 days, to issue a loan. All the values in the column that are below 180 days would be brought to the value of the minimum length of service of 180 days.

The maximum length of service is limited to reasonable limits. We are assuming that a reasonable length of service is equal to 40 years of service, that is 40 years x 365 days per year = 14600 days. Let us adjust all the values in the column that are greater than 14600 days to a value of "reasonable" experience equal to 14600.

In [21]:
# bringing the minimum and maximum values in the "days_employed" column to adequate reasonable limits.

df['days_employed'] = df['days_employed'].clip(180,14600)

**Processing borrower age column: `dob_years`**

The borrower age column has an inadequate value of "0" years. Replace this erroneous value with the next lowest value after "0" presented in our data. This value is "19".

In [22]:
# find the next minimum age value following the value "0"

df.groupby('dob_years')['children'].count().head()

dob_years
0     101
19     14
20     51
21    111
22    183
Name: children, dtype: int64

In [23]:
# replace the erroneous "0" values with a more reasonable age value of "19"

df['dob_years'] = df['dob_years'].replace(0,19)

In [24]:
df.describe(include='all')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,21525.000000,21525.000000,21525.000000,21525,21525.000000,21525,21525.000000,21525,21525,21525.000000,2.152500e+04,21525
unique,NaN,NaN,NaN,15,NaN,5,NaN,3,8,NaN,NaN,38
top,NaN,NaN,NaN,среднее,NaN,женат / замужем,NaN,F,сотрудник,NaN,NaN,свадьба
freq,NaN,NaN,NaN,13750,NaN,12380,NaN,14236,11119,NaN,NaN,797
mean,0.479721,4473.059765,43.382532,NaN,0.817236,NaN,0.972544,NaN,NaN,0.080883,1.651428e+05,NaN
std,0.755528,5123.615215,12.332333,NaN,0.548138,NaN,1.420324,NaN,NaN,0.272661,9.806554e+04,NaN
min,0.000000,180.000000,19.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,2.066726e+04,NaN
25%,0.000000,977.995667,33.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,1.074857e+05,NaN
50%,0.000000,2041.666104,42.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,1.433694e+05,NaN
75%,1.000000,5317.759754,53.000000,NaN,1.000000,NaN,1.000000,NaN,NaN,0.000000,1.955436e+05,NaN


In [25]:
# clarify which set of unique genders is present in our table

df['gender'].unique() 

array(['F', 'M', 'XNA'], dtype=object)

In [26]:
# check how many borrowers we have with a mystery gender like "XNA"

df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


There is one person with this sex in our table. We can either delete this entry or keep it. We decide to keep it, as it will not affect our calculations.

**Conclusions:**
1. Filled in the missing values in `days_employed` и `total_income`.
2. Eliminated anomalies and inadequate values in `children`, `days_employed` и `dob_years`.

### Changing data types

In [27]:
# changing data type in column `total_income` from float to integer

df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Removing duplicates.

Let's analyze the dataframe for the presence of duplicate rows. To do this, apply `duplicated()` function in combination with `sum()` to the dataframe.

In [28]:
df.duplicated().sum()

54

We found 54 duplicate rows.

Before removing the duplicate rows, let's analyze the data for implicit duplicates in the columns with string values, to see if they contain values with a different case (capital letters or a combination of capital letters and lowercase letters).

Let's start with the `education` column. Let's analyze the list of unique values present in this column.

In [29]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Noting the presence of implicit duplicates that are in different cases. To fix this, will lower all characters in the string by calling `str.lower()`.

In [30]:
df['education'] = df['education'].str.lower()
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Checked, it all worked out. There are no implicit duplicates in this column. Now let's check and eliminate implicit duplicates in other columns with string values in the same way.

In [31]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [32]:
df['family_status'] = df['family_status'].str.lower()
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [33]:
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [34]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Both columns are fine without processing.

Counting the number of duplicate rows in the data frame again.

In [35]:
df.duplicated().sum()

71

Noting that the number of duplicate rows has increased from the original estimate. Will delete all found duplicate rows, by applying `drop_duplicates()` method, deleting old index and reseting new index.

In [36]:
df = df.drop_duplicates().reset_index(drop=True)

In [37]:
df.duplicated().sum()

0

**Intermediate conclusion:**

Both explicit duplicate rows and implicit duplicates that may have appeared as a result of incorrect data collection in the row columns were found and eliminated.

### Generation of additional dataframes, decomposition of the original dataframe.

Forming a new `education_dict` dataframe by extracting the relevant columns from the original dataframe.

In [38]:
education_dict = df[['education', 'education_id']]
education_dict.head()

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1


Forming a new dataframe `family_status_dict`, by selecting the appropriate columns from the original dataframe.

In [39]:
family_status_dict = df[['family_status', 'family_status_id']]
family_status_dict.head()

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1


The resulting two tables are two separate dictionaries describing the borrower's education type and marital status, which we can subsequently refer to by identifier.

By removing these two columns with string information from the source table, we will facilitate the visual work with the source table as well as reduce the file size and data processing time of the source table.

In [40]:
df = df.drop(['education', 'family_status'], axis=1)

In [41]:
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,14600.000000,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Income categorization

For more convenient work and analysis of the borrowers' income level, the amount of total income should be grouped into certain ranges, i.e. the data should be combined into categories. To do this, we will write a function that will receive as an argument the value from the `total_income` column and, depending on its value, will assign the appropriate category, which we will save in a separate column `total_income_category`.

In [42]:
def total_income_category(total_income):
    
    '''
    Returns the income category depending on the income value, by using the following ranges:
    - 0–30000 — 'E';
    - 30001–50000 — 'D';
    - 50001–200000 — 'C';
    - 200001–1000000 — 'B';
    - 1000001 and greater — 'A'
    '''
    
    
    try: 
        if total_income <= 30000:
            return "E"
        elif 30001 <= total_income <= 50000:
            return 'D'
        elif 50001 <= total_income <= 200000:
            return 'C'
        elif 200001 <= total_income <= 1000000:
            return 'B'
        return 'A'
    
    except:
        print('Input Value Error')


In [43]:
# function check

total_income_category(5000)

'E'

In [44]:
# function check

total_income_category('100')

Input Value Error


Let's add the column `total_income_category` to the dataframe and store the corresponding borrower category equal to the value returned by our function.

To obtain category values in all rows of the dataframe, we will apply the `apply()` method to the column `total_income`, the argument of which will be the function `total_income_category` written by us.

In [45]:
df['total_income_category'] = df['total_income'].apply(total_income_category)

In [46]:
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14600.000000,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Credit purpose categorization.

For more convenient operation and analysis of the borrowing purpose, it is desirable to group the loan purpose value into specific categories. To do this, we will write a function that will receive as an argument a value from the `purpose` column and depending on its value will assign the appropriate category, which we will save in a separate column `purpose_category`.

First, let's define a unique list of credit objectives in the current dataframe.

In [47]:
df_purpose_list = df['purpose'].unique()

In [48]:
df_purpose_list

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [49]:
def purpose_category(purpose):
    
    """
    Returns the category of the loan purpose depending on the current value of the loan purpose. 
    For this purpose, in the extended purpose string we will search for the keyword by which we will 
     match one of the following categories: 
    - 'auto transactions';
    - 'real estate transactions';;
    - 'wedding';
    - 'education';       
    """
    
    try:
        if 'авто' in purpose:
            return 'операции с автомобилем'
        elif 'образован' in purpose:
            return 'получение образования'
        elif 'свадьб' in purpose:
            return 'проведение свадьбы'
        return 'операции с недвижимостью'
    
    except:
        print('Input Value Error')

In [50]:
# function check

purpose_category('сделка с подержанным автомобилем')

'операции с автомобилем'

In [51]:
# function check

purpose_category(500)

Input Value Error


Добавим в датафрейм столбец `purpose_category`, в который сохраним соответствующую категорию цели кредита, равную значению, возвращаемому нашей функцией.

Для получения значений категорий во всех строках датафрейма, применем к столбцу `purpose` метод `apply()`, в качестве аргумента которого будет служить написанная нами функция `purpose_category`.

Let's add the column `purpose_category` to the dataframe and store the corresponding borrower category equal to the value returned by our function.

To obtain category values in all rows of the dataframe, we will apply the apply() method to the column `purpose`, the argument of which will be the function `purpose_category` written by us.

In [52]:
df['purpose_category'] = df['purpose'].apply(purpose_category)

In [53]:
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14600.000000,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


### Answers to questions.

**Question 1: Is there a correlation between the number of children and repayment on time?**

To assess the relationship between the two factors, we'll apply a dataframe grouping by number of children.

In [54]:
df.groupby('children')['debt'].agg(['count', 'mean']).sort_values('mean', ascending=False).style.format({'mean':'{:.2%}'})

,count,mean
children,,
4,41,9.76%
2,2128,9.49%
1,4855,9.17%
3,330,8.18%
0,14091,7.54%
5,9,0.00%


**Conclusion:** The grouping clearly demonstrates the relationship between the presence of children and the punctuality of loan repayments. Childless borrowers have the lowest share of overdue loans, while borrowers with children have a higher percentage of overdue loans. That is, the presence of children in the family has a direct impact on the financial sustainability of the family budget.

*Note: The number of records with 3, 4 and 5 children is relatively low and unrepresentative and does not affect the overall trend sufficiently.*

**Question 2: Is there a correlation between marital status and repayment on time?**

To assess the relationship between the two factors, we'll apply a dataframe grouping by the marital status of the borrower.

In [55]:
df.groupby('family_status_id')['debt'].agg(['count', 'mean']).sort_values('mean', ascending=False).style.format({'mean':'{:.2%}'})

,count,mean
family_status_id,,
4,2810,9.75%
1,4151,9.35%
0,12339,7.55%
3,1195,7.11%
2,959,6.57%


As we have previously put the description of marital status and the description of the borrower's education level into separate dataframes: `family_status_dict` & `education_dict` respectively, we will address in our grouping directly to the separately identified table with the description of marital status.

In [56]:
df.groupby(family_status_dict['family_status'])['debt'].agg(['count', 'mean']).sort_values('mean', ascending=False).style.format({'mean':'{:.2%}'})

,count,mean
family_status,,
не женат / не замужем,2810,9.75%
гражданский брак,4151,9.35%
женат / замужем,12339,7.55%
в разводе,1195,7.11%
вдовец / вдова,959,6.57%


**Conclusion:** The grouping clearly demonstrates the relationship between marital status and repayment punctuality. Unmarried and common-law borrowers have the highest share of overdue loans, while widows/widowers have the lowest overdue repayment rate.

It is likely that widows/widowers are mature and established, and perhaps some of them are financially secured due to their age, all of which is reflected in their financial discipline.

Married people tend to have a stable and predictable financial flow, which is reflected in their share of overdue debts, which is much lower than that of people living in civil partnerships or who are not married at all.

Unmarried people tend to be younger without much experience of personal financial planning and interaction with financial institutions, which reduces their degree of compliance with repayment of credit debts.

**Question 3: Is there a correlation between income and repayment on time?**

To assess the relationship between the two factors, we apply a dataframe grouping by borrower income.

Note: When forming the income categories, we have proceeded from the following ranges of the borrower's total income:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'

In [57]:
df.groupby('total_income_category')['debt'].agg(['count', 'mean']).sort_values('mean', ascending=False).style.format({'mean':'{:.2%}'})

,count,mean
total_income_category,,
E,22,9.09%
C,16016,8.49%
A,25,8.00%
B,5041,7.06%
D,350,6.00%


**Conclusion:** The grouping clearly demonstrates the relationship between the borrower's income level and the punctuality of loan repayments.

Borrowers with the lowest income have the highest level of overdue debt. Although there are only 22 borrowers in this category, it is very difficult to remain a law-abiding borrower with income from this category.

Borrowers in category D (with an income level of 30001-50000) were the borrowers with the lowest level of delinquency. Perhaps this category, while not having a high income, is very precise and meticulous in planning its financial flows and payments and therefore is less likely to default on payments to the bank.

Predictably, Categories A and B, as the highest income borrowers, have a lower share of delinquencies, and it is likely that their delinquency is not due to financial difficulties with repayments, but to unpunctuality and inattention to repayment schedules.

**Question 4: How do the different purposes of a loan affect its repayment on time?**

To assess the relationship between the two factors, we apply a dataframe grouping by loan purpose.

Note: When generating categories for loan purposes, we grouped the extended list of loan purposes into a short list of loan purposes represented by the following categories:

- 'auto transactions';
- 'real estate transactions';;
- 'wedding arrangements';
- 'education'.

In [58]:
df.groupby('purpose_category')['debt'].agg(['count', 'mean']).sort_values('mean', ascending=False).style.format({'mean':'{:.2%}'})

,count,mean
purpose_category,,
операции с автомобилем,4306,9.36%
получение образования,4013,9.22%
проведение свадьбы,2324,8.00%
операции с недвижимостью,10811,7.23%


**Conclusion:** The grouping clearly demonstrates the relationship between the intended use of the loan and the punctuality of loan repayment.

The lowest level of delinquency is observed for loans issued for real estate transactions. This is most likely due to the fact that loans for such purposes undergo more thorough credit analysis of borrower's financial and collateral documents, which enables the bank to make deep scanning and eliminate potentially most unreliable borrowers at the stage of pre-processing of the loan application.

The opposite is true for car and education loans. These loans are usually mass express loans, which are disbursed as soon as possible without an in-depth financial evaluation of the loan application and screening of the borrower. This results in a higher level of delinquency on such loans.

## Overall conclusion

The aim of the study was to determine whether a client's marital status and number of children affects whether the loan is repaid on time.

>**The study was able to confirm the hypotheses about the relationship between the number of children and marital status on the timeliness and punctuality of loan repayments by the borrower.**
 
<u>*1. Relationship between the number of children and the default rate:*</u>

- The percentage of defaults (´delinquencies´) of borrowers with 0 children is: 7.54%
- Percentage of defaults ( delinquencies) for borrowers with "1" number of children is: 9.17%: 9.17%
- Percentage of defaults ( delinquencies) of borrowers with number of children "2" is: 9.17%: 9.49%

<u>*2. Relationship between the marital status and the default rate:*</u>

Marital status | Default rate
:-|-:
вдовец/вдова | 6.57%
в разводе | 7.11%
женат / замужем | 7.55%
гражданский брак | 9.35%
не женат/не замужем | 9.75%

---
>**In addition to the above hypotheses, the study also confirmed the hypotheses about the interdependence of the borrower's total income and different loan purposes on timely repayment of loan debt.
These dependencies are also recommended to be considered in the development of the credit scoring model**.

<u>*3. Relationship between the total income and the default rate:*</u>

Category (Income level) | Default rate
:-|-:
E (0–30000) | 9.09%
C (50001–200000) | 8.49%
A (1000001+) | 8.00%
B (200001–1000000) | 7.06%
D (30001–50000) | 6.00%

<u>*4. Relationship between the different credit purposes and the default rate:*</u>

Loan purpose | Default rate
:-|-:
операции с автомобилем |	9.36%
получение образования	|	9.22%
проведение свадьбы	|	8.00%
операции с недвижимостью	|	7.23%